<h1 align=center>Segmentación</h1>

In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import pymysql
from sqlalchemy import create_engine

Primero, hay que abrir nuestro archivo con los datos.

In [2]:
df = pd.read_csv("AccidentesAviones.csv")

¿Como solucionamos la problemática de los <h style="color: #FFAA66">"?"</h> que vimos en el notebook de introduccion? Reemplazandolos por valores nulos.

In [3]:
df.replace("?", None, inplace=True)

Ahora comienza nuestra configuración para conectar con la base de datos:

In [4]:
# Declarar la información de la configuración de MySQL en el ordenador.
usuario = "root"
host = "127.0.0.1"
contraseña = "usuario"
db= "accidentes_aviones"

# Establecer la conexión:
connection = pymysql.connect(host=host, password=contraseña, user=usuario, database=db)

In [5]:
cursor = connection.cursor()

Ahora que estamos conectados a la base da datos, hagamos la segmentación del dataframe e insertemos las tablas en MySQL.

Comencemos con la tabla principal <em>info_vuelos</em>.

In [6]:
info_vuelos = df[['Fecha',
 'Hora Declarada',
 'Ruta',
 'Operador',
 'Número de Vuelo',
 'Ruta de Vuelo',
#  'Tipo de Aeronave',
#  'Registro',
#  'CN LN',
#  'Total a Bordo',
#  'Pasajeros a Bordo',
#  'Tripulación a Bordo',
#  'Cantidad de Fallecidos',
#  'Fallecidos Pasajeros',
#  'Fallecidos Tripulación',
#  'En Tierra',
 'Resumen']].reset_index(names="indexVuelo")

Veamos como está quedando nuestra primera tabla:

In [7]:
info_vuelos.head()

,indexVuelo,Fecha,Hora Declarada,Ruta,Operador,Número de Vuelo,Ruta de Vuelo,Resumen
0,0,"September 17, 1908",1718,"Fort Myer, Virginia",Military - U.S. Army,None,Demonstration,"During a demonstration flight, a U.S. Army fly..."
1,1,"September 07, 1909",None,"Juvisy-sur-Orge, France",None,None,Air show,Eugene Lefebvre was the first pilot to ever be...
2,2,"July 12, 1912",0630,"Atlantic City, New Jersey",Military - U.S. Navy,None,Test flight,First U.S. dirigible Akron exploded just offsh...
3,3,"August 06, 1913",None,"Victoria, British Columbia, Canada",Private,None,None,The first fatal airplane accident in Canada oc...
4,4,"September 09, 1913",1830,Over the North Sea,Military - German Navy,None,None,The airship flew into a thunderstorm and encou...


Como podemos observar, las fechas están escritas, debemos transformarlas a formato fechas de la siguiente manera:

In [8]:
from datetime import datetime
fechas_datetime = []
for fecha in info_vuelos['Fecha']:
    if fecha == None:
        fechas_datetime.append(None)
        continue
    else:
        try:
            fechas_datetime.append(datetime.strptime(fecha, "%B %d, %Y"))
        except ValueError:
            print(fecha)

Ahora lo incluimos a la tabla de info_vuelos.

In [9]:
info_vuelos['fecha_procesada'] = fechas_datetime

In [10]:
info_vuelos.head(2)

,indexVuelo,Fecha,Hora Declarada,Ruta,Operador,Número de Vuelo,Ruta de Vuelo,Resumen,fecha_procesada
0,0,"September 17, 1908",1718,"Fort Myer, Virginia",Military - U.S. Army,None,Demonstration,"During a demonstration flight, a U.S. Army fly...",1908-09-17
1,1,"September 07, 1909",None,"Juvisy-sur-Orge, France",None,None,Air show,Eugene Lefebvre was the first pilot to ever be...,1909-09-07


Pongamos nombres que no contengan espacio para exportar a la base de datos SQL:

In [11]:
info_vuelos.columns = ['indexVuelo', 'fecha', 'hora_declarada', 'ruta', 'operador',
       'numero_de_vuelo', 'ruta_de_vuelo', 'resumen', 'fecha_procesada']

Hay que crear la tabla antes de exportarla:

In [14]:
query = "CREATE TABLE IF NOT EXISTS info_Vuelos (indexVuelo INT NOT NULL, fecha VARCHAR(40), hora_declarada VARCHAR(6), ruta VARCHAR(50), operador VARCHAR(50), numero_de_vuelo VARCHAR(20), ruta_de_vuelo VARCHAR(50), resumen VARCHAR(120), fecha_procesada TIME)"

cursor.execute(query)

0

Ahora podemos exportar la tabla que estuvimos modificando en este notebook.

In [15]:
engine = create_engine(f'mysql+pymysql://{usuario}:{contraseña}@{host}/{db}')

In [17]:
info_vuelos.to_sql('info_vuelos', con=engine, if_exists='replace', index=False)

5008

Continuemos con la siguiente tabla que exportaremos '<em>info_aeronave</em>'.

In [22]:
info_nave = df[[#'Fecha',
#  'Hora Declarada',
#  'Ruta',
#  'Operador',
 'Número de Vuelo',
#  'Ruta de Vuelo',
 'Tipo de Aeronave',
 'Registro',
 'CN LN']].reset_index(names= 'indexVuelo')
#  'Total a Bordo',
#  'Pasajeros a Bordo',
#  'Tripulación a Bordo',
#  'Cantidad de Fallecidos',
#  'Fallecidos Pasajeros',
#  'Fallecidos Tripulación',
#  'En Tierra',
#  'Resumen']]

In [23]:
info_nave.head(2)

,indexVuelo,Número de Vuelo,Tipo de Aeronave,Registro,CN LN
0,0,None,Wright Flyer III,None,1
1,1,None,Wright Byplane,SC1,None


Reemplacemos los nombres de las columnas:

In [25]:
info_nave.columns = ['indexVuelo', 'numero_de_vuelo', 'tipoAeronave', 'registroAeronave', 'CN_LN']

Exportemos la tabla a SQL:

In [26]:
info_nave.to_sql("info_aeronave", con=engine, if_exists='replace', index=False)

5008

Y por último trabajaremos la tabla de la tripulacion: '<em>info_tripulacion</em>'.

In [28]:
info_tripulacion = df[
    [
        # 'Fecha',
        # 'Hora Declarada',
        # 'Ruta',
        # 'Operador',
        'Número de Vuelo',
        # 'Ruta de Vuelo',
        # 'Tipo de Aeronave',
        # 'Registro',
        # 'CN LN',
        'Total a Bordo',
        'Pasajeros a Bordo',
        'Tripulación a Bordo',
        'Cantidad de Fallecidos',
        'Fallecidos Pasajeros',
        'Fallecidos Tripulación',
        'En Tierra',
        # 'Resumen'
    ]
].reset_index(names='indexVuelo')

In [29]:
info_tripulacion.head(2)

,indexVuelo,Número de Vuelo,Total a Bordo,Pasajeros a Bordo,Tripulación a Bordo,Cantidad de Fallecidos,Fallecidos Pasajeros,Fallecidos Tripulación,En Tierra
0,0,None,2,1,1,1,1,0,0
1,1,None,1,0,1,1,0,0,0


Renombramos las columnas:

In [31]:
info_tripulacion.columns = ['indexVuelo',
 'numero_de_vuelo',
 'total_a_bordo',
 'cantidadPasajeros',
 'cantidadTripulacion',
 'totalFallecidos',
 'pasajerosFallecidos',
 'tripulacionFallecidos',
 'fallecidos_en_tierra']

Ahora la incrustamos en la base de datos SQL:

In [32]:
info_tripulacion.to_sql("info_tripulacion", con=engine, if_exists='replace', index=False)

5008

<hr>
Ahora, desde nuestro motor de consultas SQL, exportamos la base de datos que creamos en formato <em>.sql</em> y trabajaremos con el en el dashboard interactivo de Power BI.
<hr>